<a href="https://colab.research.google.com/github/minyou2675/DL-study-179/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
#일반적인 Convolution layer에서 연산량은 해상도x커널크기x입출력채널개수

class MBblock(nn.Module):
  def __init__(self, in_channels, out_channels, stride = 1):
    super().__init__()
    #mobile block은 depthwise,pointwise -> batch최적화->relu 함수가 연속됨
    self.depthwise = nn.Sequential(
        nn.Conv2d(in_channels, in_channels, 3, stride = stride, padding =1,
                  groups = in_channels, bias = False),
                  nn.BatchNorm2d(in_channels),
                  nn.ReLU6(),
    ) #depth wise는 입력 해상도 x 채널 수 x 중간 featuremap 해상도
    #depthwise는 공간에 대한 convolution뿐만 아니라 채널에 대한 convolution을 통해
    #채널 압축
    self.pointwise = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 1, stride=1 , padding =0,
                  bias = False),
                  nn.BatchNorm2d(out_channels),
                  nn.ReLU6()
    ) #point wise의 연산값은 입출력 채널수 x 최종feature map 해상도
      #point wise는 공간에 대한 convolution 진행 x 채널에 대한 convolution 함

  def forward(self, x):
    x = self.depthwise(x)
    x = self.pointwise(x)
    return x
#논문에 따르면, feature map에 채널 수와 커널 크기에 의해 연산량은 결정된다.
class BasicConv2d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
    super().__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, **kwargs),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )
  def forward(self, x):
    x = self.conv(x)
    return x

class MobileNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            BasicConv2d(3, 32, 3, stride = 2, padding = 1),
            MBblock(32, 64),
            MBblock(64, 128, 2),
            MBblock(128, 128),
            MBblock(128, 256, 2),
            MBblock(256, 256),
            MBblock(256, 512, 2),
            *[MBblock(512,512) for _ in range(5)],
            MBblock(512, 1024, 2),
            MBblock(1024, 1024),
            nn.AvgPool2d(7),
            
          


        )
        self.fc = nn.Linear(1024,1000)
    
        # TODO : Implement Network
        ## Sample Convolution, delete this
        
    

    def forward(self, x):
        # TODO : Implement Network
        ## Sample Convolution, delete this
        x = self.model(x)
        
        return x

    
if __name__ == "__main__":
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')    
    
    batch_size = 4
    input = torch.randn((batch_size, 3, 224, 224))
    model = MobileNet()
    output = model(input)
    
    print(output.shape)
    
    model.to(device)
    summary(model, (3, 224, 224))

torch.Size([4, 1024, 1, 1])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             896
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
       BasicConv2d-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 32, 112, 112]             288
       BatchNorm2d-6         [-1, 32, 112, 112]              64
             ReLU6-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]           2,048
       BatchNorm2d-9         [-1, 64, 112, 112]             128
            ReLU6-10         [-1, 64, 112, 112]               0
          MBblock-11         [-1, 64, 112, 112]               0
           Conv2d-12           [-1, 64, 56, 56]             576
      BatchNorm2d-13           [-1, 64, 56, 56]             128
           